In [ ]:
import importlib
import datetime
import random

import pandas as pd
import numpy as np
from geopy import distance as gp_distance
import haversine

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=[],
)

### Define functions

In [ ]:
def convert_to_datetime(dttm_str):
    return datetime.datetime.strptime(dttm_str, '%Y-%m-%d %H:%M:%S')

def choose_random_time_in_interval(dttm_left, dttm_right):
    delta = (dttm_right - dttm_left).total_seconds()
    return dttm_left + datetime.timedelta(seconds=random.randint(0, delta))

def parse_gps_str(gps_str):
    return tuple(float(c) for c in gps_str.split(','))

def assign_day(dttm, month, day):
    return dttm.replace(month=month, day=day)

### Get and prepare dataframes

In [ ]:
market_orders_df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/moscow_deliveries_2020-10-29_2020-11-04')
    .as_dataframe()
)
market_orders_df = market_orders_df[~pd.isnull(market_orders_df['final_delivery_point_gps'])]

In [ ]:
market_orders_df.head(2)

In [ ]:
# Market Orders.
market_orders_df['original_delivery_from_datetime'] = (
    market_orders_df['original_delivery_from_datetime'].apply(
        lambda x: x[:10] + ' ' + '09:00:00' if x[11:] < '09:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 10, 29)
    )
)
market_orders_df['original_delivery_to_datetime'] = (
    market_orders_df['original_delivery_to_datetime'].apply(
        lambda x: x[:10] + ' ' + '23:00:00' if x[11:] > '23:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 10, 29)
    )
)

market_deliveries_df = pd.DataFrame()
market_deliveries_df['service_name'] = ['market']*len(market_orders_df)
market_deliveries_df['weight'] = market_orders_df['order_physical_weight'].fillna(
    np.mean(market_orders_df['order_physical_weight'])
)
market_deliveries_df['volume'] = None
market_deliveries_df['delivery_lat'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[1])
)
market_deliveries_df['delivery_lon'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[0])
)
market_deliveries_df['warehouse_lat'] = None
market_deliveries_df['warehouse_lon'] = None
market_deliveries_df['warehouse_id'] = None
market_deliveries_df['delivery_plan_dttm'] = market_orders_df.apply(
    lambda x: choose_random_time_in_interval(
        x['original_delivery_from_datetime'],
        x['original_delivery_to_datetime']
    ),
    axis=1
)
market_deliveries_df['original_delivery_from_datetime'] = market_orders_df.apply(
    lambda x: assign_day(x['original_delivery_from_datetime'], 10, 29),
    axis=1
)
market_deliveries_df['original_delivery_to_datetime'] = market_orders_df.apply(
    lambda x: assign_day(x['original_delivery_to_datetime'], 10, 29),
    axis=1
)
market_deliveries_df['order_id_source'] = market_orders_df['order_id'].astype(int)
market_deliveries_df['warehouse_ready_dttm'] = None
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lat'])]
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lon'])]

In [ ]:
market_deliveries_df.shape

In [ ]:
market_deliveries_df = market_deliveries_df[market_deliveries_df['weight']<=10]

In [ ]:
market_deliveries_df.shape

In [ ]:
# market_deliveries_df.head()

In [ ]:
market_orders_df.shape

### Calculate fact CPO for market

In [ ]:
PARTNER_CPO_FIELDS = [
    'partner_CPO_delivery',
    'partner_CPO_insurance',
    'partner_CPO_lift',
    'partner_CPO_other',
    'partner_CPO_return',
    'partner_CPO_sort',
    'partner_CPO_total_sum',
]
OWN_CPO_FIELDS = [
    'own_CPO_PVZ_lavka_locker_storage',
    'own_CPO_SC',
    'own_CPO_delivery_from_PVZ_lavka_locker_to_customer',
    'own_CPO_delivery_from_SC_to_PVZ_lavka_locker',
    'own_CPO_delivery_from_SC_to_customer',
    'own_CPO_zabor_from_supplier_to_SC',
]

In [ ]:
orders = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/regional_deliveries_2020-10-29_2020-11-04')
    .as_dataframe()
)

In [ ]:
orders.iloc[:1]

In [ ]:
orders['total_delivery_cpo'] = orders[
    OWN_CPO_FIELDS + ['partner_CPO_total_sum']
].sum(axis=1)

In [ ]:
(
    orders['total_delivery_cpo'].sum()
    / (orders['total_delivery_cpo']!=0).sum()
)

In [ ]:
mean_cpo['regions_no_spb_msk'] = {}
for col in (PARTNER_CPO_FIELDS+OWN_CPO_FIELDS+['total_delivery_cpo']):
    mean_cpo['regions_no_spb_msk'][col] = round(
        (orders[col].sum())
        / (orders[(orders[col]!=0) & (~pd.isnull(orders[col]))].shape[0]+1e-7) ,
        1
    )

In [ ]:
pd.DataFrame(mean_cpo).T.to_excel('./analytics/cpo_market_real_data.xlsx')